# Α profitable museum heist

Also known as the the 0/1 knapsack problem

---

**Disclaimer:** *While I enjoy using engaging examples like "museum heists" to introduce dynamic programming and the 0/1 knapsack problem, I must firmly state that neither the department, nor Loyola University Chicago, encourages, condones, or tolerates actual museum heists, art theft, or any other illegal activities involving priceless artifacts, well-guarded vaults, or overly dramatic laser grids. These examples are purely for educational purposes and designed to help you solve optimization problems in code—not in real life. Any resemblance to actual heists, either past or future, is purely coincidental. Please use your algorithms responsibly.*

---

**A more serious warning:** In this notebook the symbol $\mathcal{S}$, a calligraphic rendering of S, represents a set. The plain symbol $S$ represents the value of the contents of that set.

---


## Problem Setup

Consider a set $ M = \{1, 2, \ldots, n\} $ of $ n $ items, where each item $ i \in M $ has a positive value $ v_i $ and a positive weight $ w_i $. The goal is to find a subset $ \color{magenta}{\mathcal{S}'} \subseteq M $ whose items add to the highest value while their total weight does not exceed a given limit $ C_\max > 0 $. This can be formulated as:

$$
\color{magenta}{\mathcal{S}'} =\arg\max_\mathcal{S} \sum_{i \in \mathcal{S}} v_i \quad \text{subject to} \quad \sum_{i \in \color{magenta}{\mathcal{S}'}} w_i \leq C_\max
$$

For example, imagine a truck with a weight capacity $ C_\max $ that we use to transport items.

Brute force can solve this problem for small $ n $. For instance, if $ n = 10 $, we can evaluate each of the $ 2^{10} $ subsets, discard those exceeding the weight limit, and select the one with the highest value. However, as $ n $ grows, this approach quickly becomes infeasible.

Imagine a scenario where a group of art thieves targets the Art Institute of Chicago, planning to steal items using a truck with a weight limit of $ C_\max = 500 $ lbs. With nearly 300,000 artifacts in the museum, there are $ 2^{300,000} $ possible combinations, roughly $ 10^{100,000} $. Even if they could evaluate one billion combinations per second, they would only manage about $ 10^{17} $ combinations per year. It would take them approximately $ 10^{99,983} $ years to check all possibilities—far longer than the age of the universe, which is about $ 1.4 \times 10^{10} $ years.



## Can We Help the Thieves?  
*(Yes, you are paying tuition for this!)*

What if we told the would-be thieves that we could find the best combination in just 150 million steps, instead of the astronomical $10^{100,000}$ steps? They could have their answer in just 0.15 seconds (since we can compute 1 billion combinations per second) rather than waiting an eternity—roughly $10^{99,983}$ years!

Wouldn't that be worth something? Maybe 20% of their profits?



<center>

![American Gothic](https://drive.google.com/uc?export=view&id=1dUo-zLokDVRIKpSsZ4_3HROPVvDWwDlQ)
![Nighthawks](https://drive.google.com/uc?export=view&id=1dY4ecE6WZWNajY-l_ZDENj_igRGVsSUL)
![Sky Above Clouds IV](https://drive.google.com/uc?export=view&id=1dTqECkj9HV0Vm3SicGNheGptw3fl_4Mf)
![Old Guitarist](https://drive.google.com/uc?export=view&id=1dQU88AaWqTkAFu-_BLGkmDJvYsy5dL6E)

*Four emblematic paintings in the Art Institute of Chicago: American Gothic by Grant Wood; Nighthawks by Edward Hopper; Sky Above Clouds by Georgia O'Keefe, and; The Old Guitarist by Pablo Picasso. The pictures have been cropped to fit in this notebook.*
</center>



## How?

Instead of considering all the artifacts, let’s narrow the focus to the set $ M $ of paintings in the Art Institute:

$$
M =
\{
  \textsf{American Gothic},\ \
  \textsf{Nighthawks},\ \
  \textsf{Sky Above Clouds IV},\ldots,\  
  \textsf{The Old Guitarist}
\}
$$

The museum has about 3,000 paintings, which means there are $ 2^{3000} $ possible combinations. At least one of these combinations weighs around 500 lbs and is the most valuable set the thieves could carry.

Now, suppose we've somehow found the optimal solution: a subset $ S $ of $ M $ containing the most valuable paintings without exceeding the 500 lb limit.

Here’s the key question: **Could the last painting in** $ M $—Picasso’s *Old Guitarist*—**be part of the solution or not?** When the thieves drive away with their loot, is *The Old Guitarist* in the truck?


### Picasso's *Old Guitarist* **could not** be in the truck

The thieves drive away with the same loot as if the painting was never part of the museum's collection. In other words, the most valuable subset of

$$
M =
\{
  \textsf{American Gothic},\ \
  \textsf{Nighthawks},\ \
  \textsf{Sky Above Clouds IV},\ldots,\  
  \textsf{The Old Guitarist}
\}
$$

that fits in a 500 lbs truck, is also the most valuable subset of

$$M-\{\textsf{The Old Guitarist} \}$$<br/>

### Picasso's *Old Guitarist* **could be** in the truck

Right before grabbing *The Old Guitarist,* the thieves had found the most valuable subset of
$$M-\{\textsf{The Old Guitarist} \}$$
to fit in their truck, now fully loaded with 500 lbs of paintings. When they come across the last painting, they face a dilemma: drive away with what they already got or unload a few
paintings to make room and take the Picasso with them?

If they unload a few paintings, the capacity of the truck becomes $$500-w_\textsf{The Old Guitarist}$$
i.e. enough to fit the Picasso painting. For this new capacity, there may exist a different subset of $M-\{\textsf{The Old Guitarist} \}$ that is the most valuable combination of paintings with total weight $\leq 500-w_\textsf{The Old Guitarist}$.

Is the value of this different subset, plus the value of the Picasso painting, higher or lower than the value of previous subset?

### Notation

We need some notation to differentiate between the values of two subsets mentioned above.

One is the most valuable subset with weight $\leq 500$ lbs that does not include the last painting (in this example, *The Old Guitarist).* We denote the value of this subset as

$$
\underbrace{
S(
  \underbrace{n-1}_{
    \begin{array}{r}
    \text{exclude}\\
    \text{the last}\\
    \text{painting}
    \end{array}
  },
  \underbrace{C_{\max}}_{
    \begin{array}{l}
    \text{load the}\\
    \text{truck to}\\
    \text{capacity}
    \end{array}
  }
  ) \\
}_{
  \begin{array}{c}
  \text{Maximum value when the truck is fully}\\
  \text{loaded but without the last painting.}
  \end{array}
}
$$



where $n$ is the last item in the collection, therefore $n-1$ shows that we exclude it, and $C_\max$ is the maximum loading capacity of the truck (500 lbs in our example).

The other subset is the most valuable assortment of paintings we can fit in $C_\max-w_n$ plus the last painting. We write the value of this subset as

$$
\underbrace{S(
  \underbrace{n-1}_{
    \begin{array}{r}
    \text{exclude}\\
    \text{the last}\\
    \text{painting}
    \end{array}
  },
  \underbrace{C_{\max} -w_n}_{{
    \begin{array}{l}
    \text{leave room in}\\
    \text{the truck for}\\
    \text{last painting}
    \end{array}
  }}
  )
  +
  \underbrace{v_n}_{
    \begin{array}{c}
    \text{then add} \\
    \text{last painting}\\
    \text{to truck}
    \end{array}
  } \\
}_{
  \begin{array}{c}
  \text{Value when the truck is partially loaded}\\
  \text{with the best deal possible that allows room for}\\
  \text{the last painting, plus the value of the last painting}
  \end{array}
}
$$

where $v_n$ is the value of the last painting and $w_n$ is its weight.

### Take it or leave it

The decision to take or leave the last painting comes to a simple comparison between two options.


#### Leave it

There is no room in the truck for the last painting. The truck is full to capacity and the items we take are the most valuable subset of $M-\{n\}$. The total value of this option is $S(n-1, C_\max)$.

#### Take it

We make room in the truck for the last painting. Before we load the last painting, the items we take are the most valuable subset of $M-\{n\}$ that fit in the reduced capacity of the truck. The value of that subset is $S(n-1, C_\max-w_n)$. When we add the last item, the truck is at capacity and the total value of this option is $\color{maroon}{v_n}+S(n-1, C_\max-w_n)$

### Well, take it or leave it?

It comes down to a simple comparison: if

\begin{align}
S(n-1, C_\max) > \color{maroon}{v_n}+S(n-1, C_\max-w_n)
\end{align}

leave the $n$-th item.

### Implications of $n\not\in \mathcal{S}$

If the last item $n$ **is not** in the optimal solution $S$, then $S$ is an optimal solution to the problem with $\{1,2,\ldots n-1\}$ items as well. We can prove it mathematically (see Miscellaneous section at the end), but first let's agree that it's just common sense.

### Implications of $n\in \mathcal{S}$

If the last item $n$ *could* in the optimal solution $S$, then we know that $S$ **cannot be** an optimal solution for the smaller set $\{1,2,\ldots,n-1\}$. Why? Because the smaller set does not include $n$. But $n$ is in the optimal solution $n$, which means that $S$ must be a subset of $\{1,2,\ldots,n\}$.

If we removed $n$ from $\mathcal{S}$, then $\mathcal{S}-\{n\}$ is an optimal solution for $\{1,2,\ldots,n-1\}$. The total weight of $\mathcal{S}-\{n\}$ is

$$
\sum_i w_i \leq C_\max-w_n,\ \ i\in \mathcal{S}-\{n\}
$$

and its total value is $V-v_n$, where

$$
V = \sum_{i\in \mathcal{S}} v_i
$$
is the total value of items in $\mathcal{S}$.

The reduction in weight above, by $w_n$, allows room to add the last item $n$ to $\mathcal{S}-\{n\}$. The question now is: **do we want to do it?**

Since $\mathcal{S}-\{n\}$ is an optimal solution for the smaller problem of $\{1,2,\ldots,n-1\}$, it has the best possible value among all subsets of $\{1,2,\ldots,n-1\}$ whose weight is at-or-below $C_\max-w_n$. We denote that value as

$$S(n-1, C_\max-w_n)$$

(drawing attention to the two different symbols with similar appearance: $\mathcal{S}$ is the subset containing the most valuable items for a given weight restriction, while $S$ is the value of those items).

There is room for the last item $n$. If we add that item to $\mathcal{S}-\{n\}$, the total value becomes

$$S(n-1, C_\max-w_n)\ + v_n$$

Just because we can carry $w_n$ weight however, doesn't mean that we have to take item $n$. Are we better of`f by taking other items whose weight comes at-or-below $w_n$ and whose value exceeds the added value of $n$?

We decide this matter by comparing the values of our two options:

$$
S(n,C_\max) =
\max
\left\{
  \begin{array}{l}
  S(n-1, C_\max), \\
  S(n-1, C_\max-w_n)+v_n
  \end{array}
\right\}
$$

## Formal stuff

The formula above gives the value of the optimal solution for the full set $\{1,2,\ldots, n\}$. The items included in this solution have a total weight at-or-below our maximum capacity $C_\max$.

The same principle applies however when we try to find the optimal capacity for smaller problems like  $\{1,2,\ldots, \color{green}{n-1}\}$, $\{1,2,\ldots, \color{green}{n-2}\}$, etc. Essentially, we are solving a problem with fewer items and different capacities. In general, we write

\begin{align}
 S(i, r)  = &
\begin{cases}
\color{maroon}{
S(i-1,r)},\ &\text{when}\ w_i > r\\
\max\left\{\color{maroon}{S(i-1,r),}\ S(i-1,r-w_i)+v_i\right\},\ &\text{when}\ w_i \leq r
\end{cases}\\ \\
& 0 \leq  i \leq n  \\
& 0 \leq r \leq C_\max
\end{align}

where $r$ is the *residual capacity*, i.e., how much capacity is available after we take on some load.

It is straight-forward to determine the initial values of $S(i,r)$:

\begin{align}
S(0,r) & = 0\ \ (\text{There are 0 items to chose from}) &\\
S(i,0) & = 0\ \ (\text{There is no room to take aything}) &
\end{align}

With these initial values, we can begin to build the rest of the values, for example:

\begin{align}
 S(\color{brown}1, \color{blue}1)  = &
\begin{cases}
S(0,\color{blue}1),\ &\text{if}\ w_\color{brown}1 > \color{blue}1\\
\max\left\{S(0,\color{blue}1),\ S(0,\color{blue}1-w_\color{brown}1)+v_\color{brown}1\right\},\ &\text{if}\ w_\color{brown}1 \leq \color{blue}1
\end{cases}
\end{align}

This probably evalues to 0 if $w_1 > 1$ or $v_1$ if $w_1\leq 1$. In other words if the weight of the first item is 1 (whatever weight units we use), we can take it and add its value $v_1$ to $S$.

After $S(1,1)$ we can compute

\begin{align}
 S(\color{brown}1, \color{blue}2)  = &
\begin{cases}
S(0,\color{blue}2),\ &\text{if}\ w_\color{brown}1 > \color{blue}2\\
\max\left\{S(0,\color{blue}2),\ S(0,\color{blue}2-w_\color{brown}1)+v_\color{brown}1\right\},\ &\text{if}\ w_\color{brown}1 \leq \color{blue}2
\end{cases}
\end{align}

and so on.

## Danger ahead!

At this point we can imagine that given weight and value vectors
$(w_1, w_2,\ldots,w_n)$
and
$(v_1, v_2,\ldots,v_n)$,
the  pseudocode will be like so:

\begin{align}
& S(0,r) \leftarrow 0,\ 0\leq r\leq C_\max{} \\
& S(i,0) \leftarrow 0,\ 0\leq i\leq n \\
& \quad \text{for}\ \color{brown}i = 1 \ldots n \\
& \quad\quad \text{for}\ \color{blue}r=1\ldots C_\max{} \\
& \quad\quad\quad S(\color{brown}i,\color{blue}r) \leftarrow S(\color{brown}i-1,\color{blue}r) \\
& \quad\quad\quad \text{if}\ w_\color{brown}i \leq \color{blue}r\\
& \quad\quad\quad\quad S(\color{brown}i,\color{blue}r) \leftarrow \max{\left\{S(\color{brown}i-1,\color{blue}r),\ v_\color{brown}i+S(i-1,\color{blue}r-w_\color{brown}i)\right\}} \\
& \text{return}\ S
\end{align}

The algorithm above returns matrix $S$, whose bottom right element $S(n,C_\max)$ is the value of the most valuable subset of $\{1,2,\ldots,n\}$ that can fit at or below $C_\max$. **We still don't know what the contents of that subset are!** In other words, we know that we can drive away from the Art Institute with a couple of billion dollars worth fo art in our truck, we just don't know **yet** which pieces of art to load to the truck.

## Implementation

To implement the <strike>Knapsack</strike> Truck problem we need to decide what data structures to use. The data needed for this problem are the values and the weights of the items we wish to consider. The simplest approach is to use two arrays, one for values and one for weights. The arrays must be synchronized, so that the `i`-th element in both arrays related to the same item.

The length of the arrays must be the same. From that length we can tell how many items there are.

We assume that values and weights are integer numbers. The desired output is the table with all $S(i,r)$ values.

One issue to pay attention to is the array referencing. The algorith enumerates item beginning with 1, not 0. The weight of the first item is $w_\color{red}1$ and its value is $v_\color{red}1$. In 0-based array notation we use $\texttt w[\color{red}0]$ and $\texttt v[\color{red}0]$ for the same information. Correspondigly, the last item ($w_\color{red}n$, $v_\color{red}n$) would be $\texttt w[\color{red}{n-1}]$ and $\texttt v[\color{red}{n-1}]$.

In [ ]:
def simple_truck(v, w, c_max):
  # arrays must have same length
  if len(v) != len(w):
    return None
  # Number of items
  n = len(v)
  # Create output array with n + 1 rows and as many rows as max capacity + 1
  # The +1 is to allow room for the S(0,r) and S(i,0) scenarios. The
  # initialization below takes care of S(0,r) = 0 and S(i,0) = 0 assignments.
  S = [[0 for _ in range(1+c_max)] for _ in range(1+n) ]
  # Explore non trivial cases
  for i in range(1, n+1):
    for r in range(1, c_max+1):
      # The most valuable subset among the first i items in the collection whose
      # total weight is at-or-below r has value S(i,r).
      if w[i-1] > r:
        # i-th item cannot be included in subset, use immediately smaller subset
        # We index to i-1 because values and weights are plain lists, and their
        # first elements are in [0]
        S[i][r] = S[i-1][r]
      else:
        # If there is room for the i-th item, it is more profitable to take it
        # or leave it?
        S[i][r] = max(
            S[i-1][r],
            v[i-1]+S[i-1][r-w[i-1]])
  return S

# A simple test
values = [5,4,3,2]
weights = [4,3,2,1]
c_max = 6
S = simple_truck(values, weights, c_max)
S

[[0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 5, 5, 5],
 [0, 0, 0, 4, 5, 5, 5],
 [0, 0, 3, 4, 5, 7, 8],
 [0, 2, 3, 5, 6, 7, 9]]

## Unraveling ${S}$

The typical assignment asks students to build the set $\mathcal S$ given the code for matrix $S$.

How did we get to $S(n,C_\max)$? According to the algorithm, earlier,

\begin{align}
 S(\color{brown}n, \color{blue}{C_\max})  = &
\begin{cases}
S(\color{brown}n-1,\color{blue}{C_\max}),\ &\text{if}\ w_\color{brown}n > \color{blue}{C_\max}\\
\max\left\{S(\color{brown}n-1,\color{blue}{C_\max}),\ S(\color{brown}n-1,\color{blue}{C_\max}-w_\color{brown}n)+v_\color{brown}n\right\},\ &\text{if}\ w_\color{brown}n \leq \color{blue}{C_\max}
\end{cases}
\end{align}

The first case, $w_\color{brown}n > \color{blue}{C_\max}$, applies if item $\color{brown}n$ is too big to carry even with an empty truck.

If item $\color{brown}n$ fits in the truck, do we take it or not? To decide that we look at

$$
\max\left\{S(\color{brown}n-1,\color{blue}{C_\max}),\ S(\color{brown}n-1,\color{blue}{C_\max}-w_\color{brown}n)+v_\color{brown}n\right\}
$$

Specifically, if
$$
S(\color{brown}n, \color{blue}{C_\max}) =
S(\color{brown}n-1,\color{blue}{C_\max})
$$

leave $\color{brown}n$, else if

$$
S(\color{brown}n, \color{blue}{C_\max}) = S(\color{brown}n-1,\color{blue}{C_\max}-w_\color{brown}n)+v_\color{brown}n
$$

take $\color{brown}n$.



## Constructing $\mathcal{S}$

We begin with an empty set, $\mathcal{S}=\varnothing$, and traverse matrix $S$ from its bottom-right cell.

\begin{align}
& \mathcal{S}\leftarrow\varnothing \\
& r \leftarrow C_\max \\
& \text{for}\ i = n\ldots 1 \\
& \quad \text{if}\ S(i,r) = S(i-1,r-w_i)+v_i \\
& \quad\quad \mathcal{S} \leftarrow \mathcal{S}\cup \{i\} \\
& \quad\quad r\leftarrow r-w_i \\
& \text{return } \mathcal S
\end{align}

The loop considers every item in the collection $M$ and determines if it should be added to the most valuable subset $\mathcal S\subseteq M$. The $i$-th item is added to $\mathcal S$ when the second term below, wins.
$$
\max\left\{\color{gray}{S(i-1,{r})},\
S(\color{brown}i-1,\color{blue}{r}-w_\color{brown}i)+v_\color{brown}i
\right\} \
$$
When the *if*-statement evaluates to true, the $i$-th item is added to $\mathcal S$. The available capacity is reduced by $\color{blue}{r}-w_\color{brown}i$ units and we proceed with the next item $i-1$.

On the other hand, when the winning term is
$$
\max\left\{{S(\color{brown}i-1,\color{blue}{r})},\ \color{gray}{
S(i-1,{r}-w_i)+v_i}
\right\}\
$$
the *if* statement evaluates to false and $i$-th item is not added to $\mathcal S$. The capacity $r$ remains the same and we proceed with the next item $i-1$, and so on.

In [ ]:
def optimal_subset(v, w, S):
  subset = list()
  total_w = 0
  total_v = 0
  r = len(S[0])-1 # C_max
  n = len(S)-1
  for i in range(n, 0, -1):
    # Remember to reference w[] and v[] by -1 to compensate for
    # 0-based indexing
    if S[i][r] == S[i-1][r-w[i-1]] + v[i-1]:
      subset.append(i)
      r -= w[i-1]
      total_w += w[i-1]
      total_v += v[i-1]
  return subset, total_v, total_w

subset = optimal_subset(values, weights, S)



([4, 3, 2], 9, 6)

In [ ]:
# Another simple test
values = [5,11,10,20,6,15]
weights = [1,5,4,10,1,4]
c_max = 12
S = simple_truck(values, weights, c_max)
S

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
 [0, 5, 5, 5, 5, 11, 16, 16, 16, 16, 16, 16, 16],
 [0, 5, 5, 5, 10, 15, 16, 16, 16, 21, 26, 26, 26],
 [0, 5, 5, 5, 10, 15, 16, 16, 16, 21, 26, 26, 26],
 [0, 6, 11, 11, 11, 16, 21, 22, 22, 22, 27, 32, 32],
 [0, 6, 11, 11, 15, 21, 26, 26, 26, 31, 36, 37, 37]]

## Reading material

* [Dynamic Programming](https://jeffe.cs.illinois.edu/teaching/algorithms/book/03-dynprog.pdf) (Chapter 3 from Jeff Erickson's book).
* [Dynamic Programming](https://web.mit.edu/15.053/www/AMP-Chapter-11.pdf) (Chapter 11, from Applied Mathematical Programming, by Bradley, Hax, and Magnanti).
* [The Theory of Dynamic Programming](https://www.rand.org/content/dam/rand/pubs/papers/2008/P550.pdf) Richard Bellman's 1954 paper summarizing his work.
* [Dynamic Programming](https://people.eecs.berkeley.edu/~vazirani/algorithms/chap6.pdf) (Chapter 6 from Algorithms by Dasgupta, Papadimitriou, and Vazirani).'
* Leo's slide deck on [the anatomy of a dynamic programming table](https://docs.google.com/presentation/d/1fhhKnA9CH3AY_ltPt4qgtjsXocscWCf5C2cgxi4RCKw/edit?usp=sharing).

## Miscellaneous

### Proofs

**Theorem:** if S is an optimal solution for $\{1,2,\ldots,\color{maroon}{n-1,} \color{red}n\}$ and  $\color{red}n\not\in S$, then $S$ is an optimal solution for $\{1,2,\ldots,\color{maroon}{n-1}\}$ as well.

If $S$ is an optimal solution, then
$$\sum_{i\in s}w_i \leq C_\max$$
Let $V=\sum_{i\in S}v_i$ be the value of all items in $S$. Because $S$ is an optimal solution for $\{1,2,\ldots,\color{maroon}{n-1}\}$, it means that $V$ is the highest possible value of any subset of $\{1,2,\ldots,\color{maroon}{n-1}\}$ that meets the weight restriction.

**Proof.** Assume that there is another set $S^*$ that is the optimal solution for $\{1,2,\ldots,\color{maroon}{n-1}\}$. Its value is $V^* = \sum_{i\in S^*}v_i$, such that $V^*>V$. But this contradicts the assumption that $S$'s value is the highest possible value of any subset of $\{1,2,\ldots,\color{maroon}{n-1}\}$, therefore, $S^*$ cannot be an optimal subset.